In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Stored in directory: C:\Users\Nick\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [7]:
!pip install keras

In [10]:
!pip install tensorflow

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [11]:
# Set features. This will also be used as your x values.
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

selected_features = df.values[:, 1:40]
y = df.values[:, 0]

label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
#[1,0,0]=candidate
#[0,1,0]=confirmed
#[0,0,1]=false positive

# Create a Train Test Split

Use `koi_disposition` for the y values

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, one_hot_y, random_state=29)
X_encoded_train, X_encoded_test, y_encoded_train, y_encoded_test = train_test_split(selected_features, encoded_y, random_state=29)

In [14]:
X_train

array([[0, 0, 0, ..., -0.079, 292.83215, 48.219359999999995],
       [1, 0, 0, ..., -0.069, 298.53922, 41.986961],
       [1, 0, 0, ..., -0.805, 299.57739, 40.458481],
       ...,
       [0, 0, 0, ..., -0.071, 293.2244, 40.931931],
       [0, 0, 0, ..., -0.782, 291.17792000000003, 39.199490000000004],
       [0, 0, 0, ..., -0.126, 291.92255, 42.002559999999995]],
      dtype=object)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [27]:
# Scale your data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_encoded_scaler = StandardScaler().fit(X_encoded_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_encoded_train_scaled = X_encoded_scaler.transform(X_train)
X_encoded_test_scaled = X_encoded_scaler.transform(X_test)

C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype object was converted

# Train the Model



In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=39))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    X_train_scaled,
    y_train,
    epochs=250,
    shuffle=True,
    verbose=2
)




Train on 5243 samples
Epoch 1/250
5243/5243 - 1s - loss: 0.9375 - accuracy: 0.4927
Epoch 2/250
5243/5243 - 0s - loss: 0.6733 - accuracy: 0.6975
Epoch 3/250
5243/5243 - 0s - loss: 0.4773 - accuracy: 0.7955
Epoch 4/250
5243/5243 - 0s - loss: 0.3887 - accuracy: 0.8241
Epoch 5/250
5243/5243 - 0s - loss: 0.3440 - accuracy: 0.8440
Epoch 6/250
5243/5243 - 0s - loss: 0.3156 - accuracy: 0.8575
Epoch 7/250
5243/5243 - 0s - loss: 0.2963 - accuracy: 0.8724
Epoch 8/250
5243/5243 - 0s - loss: 0.2846 - accuracy: 0.8758
Epoch 9/250
5243/5243 - 0s - loss: 0.2782 - accuracy: 0.8817
Epoch 10/250
5243/5243 - 0s - loss: 0.2730 - accuracy: 0.8848
Epoch 11/250
5243/5243 - 0s - loss: 0.2688 - accuracy: 0.8873
Epoch 12/250
5243/5243 - 0s - loss: 0.2654 - accuracy: 0.8877
Epoch 13/250
5243/5243 - 0s - loss: 0.2634 - accuracy: 0.8896
Epoch 14/250
5243/5243 - 0s - loss: 0.2609 - accuracy: 0.8915
Epoch 15/250
5243/5243 - 0s - loss: 0.2592 - accuracy: 0.8909
Epoch 16/250
5243/5243 - 0s - loss: 0.2570 - accuracy: 0.

Epoch 133/250
5243/5243 - 0s - loss: 0.2191 - accuracy: 0.9058
Epoch 134/250
5243/5243 - 0s - loss: 0.2185 - accuracy: 0.9088
Epoch 135/250
5243/5243 - 0s - loss: 0.2191 - accuracy: 0.9044
Epoch 136/250
5243/5243 - 0s - loss: 0.2190 - accuracy: 0.9054
Epoch 137/250
5243/5243 - 0s - loss: 0.2193 - accuracy: 0.9062
Epoch 138/250
5243/5243 - 0s - loss: 0.2186 - accuracy: 0.9054
Epoch 139/250
5243/5243 - 0s - loss: 0.2194 - accuracy: 0.9060
Epoch 140/250
5243/5243 - 0s - loss: 0.2188 - accuracy: 0.9064
Epoch 141/250
5243/5243 - 0s - loss: 0.2186 - accuracy: 0.9046
Epoch 142/250
5243/5243 - 0s - loss: 0.2183 - accuracy: 0.9052
Epoch 143/250
5243/5243 - 0s - loss: 0.2183 - accuracy: 0.9067
Epoch 144/250
5243/5243 - 0s - loss: 0.2191 - accuracy: 0.9062
Epoch 145/250
5243/5243 - 0s - loss: 0.2184 - accuracy: 0.9052
Epoch 146/250
5243/5243 - 0s - loss: 0.2184 - accuracy: 0.9075
Epoch 147/250
5243/5243 - 0s - loss: 0.2185 - accuracy: 0.9056
Epoch 148/250
5243/5243 - 0s - loss: 0.2182 - accuracy:

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [29]:
# Create the GridSearchCV model
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
model2 = SVC(kernel='linear')
model2.fit(X_encoded_train_scaled,y_encoded_train)
print(f"Training Data Score: {model2.score(X_encoded_train_scaled, y_encoded_train)}")
print(f"Testing Data Score: {model2.score(X_encoded_test_scaled, y_encoded_test)}")

Training Data Score: 0.8947167652107572
Testing Data Score: 0.8804347826086957


In [32]:
# Train the model with GridSearch
param_grid = {'C': [.01, .1, 1, 10, 100],
              'gamma': [0.0001, 0.001, 0.01]}
grid2 = GridSearchCV(model2, param_grid, verbose=3)
grid2.fit(X_encoded_train_scaled, y_encoded_train)

C:\Users\Nick\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV] C=0.01, gamma=0.0001 ............................................
[CV] .... C=0.01, gamma=0.0001, score=0.858776443682104, total=   0.3s
[CV] C=0.01, gamma=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] ... C=0.01, gamma=0.0001, score=0.8672768878718535, total=   0.2s
[CV] C=0.01, gamma=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


[CV] ... C=0.01, gamma=0.0001, score=0.8654066437571593, total=   0.3s
[CV] C=0.01, gamma=0.001 .............................................
[CV] ..... C=0.01, gamma=0.001, score=0.858776443682104, total=   0.3s
[CV] C=0.01, gamma=0.001 .............................................
[CV] .... C=0.01, gamma=0.001, score=0.8672768878718535, total=   0.2s
[CV] C=0.01, gamma=0.001 .............................................
[CV] .... C=0.01, gamma=0.001, score=0.8654066437571593, total=   0.2s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] ...... C=0.01, gamma=0.01, score=0.858776443682104, total=   0.3s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] ..... C=0.01, gamma=0.01, score=0.8672768878718535, total=   0.2s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] ..... C=0.01, gamma=0.01, score=0.8654066437571593, total=   0.2s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.01, 0.1, 1, 10, 100], 'gamma': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [33]:
print(grid2.best_params_)
print(grid2.best_score_)

{'C': 10, 'gamma': 0.0001}
0.8928094602326913


# Save the Model

In [34]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
model.save("Star_Platinum.h5")